# Session 1.1: Loading and Manipulating Your Molecular Simulation Data

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Author: Dr Richard Gowers   
Email: richardjgowers@gmail.com

## Learning outcomes:
* How to load your molecular simulation data into MDAnalysis
* Basic features of the `MDAnalysis.Universe`
* Visualise your simulation data with `nglview`
* Working with `AtomGroup`s
* Select atoms of interest using `select_atoms`
* Selecting bonds
* Adding extra information to a `Universe`


**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

## 1. Loading your data

## Next Notebook

[Working with your coordinates](MD_02_Geometry.ipynb)